# Running Python, R or C Code from Julia

This notebook provides a basic introduction to how to run Python, R and C code from Julia.

In [1]:
using Printf, DelimitedFiles
include("jlFiles/printmat.jl")

printyellow (generic function with 1 method)

# Load Data

In [2]:
x = readdlm("Data/MyData.csv",',',skipstart=1)  #reading the csv file

(Rme,Rf,R) = (x[:,2],x[:,3],x[:,4])  #creating variables from columns of x
y  = R - Rf                          #do R .- Rf if R has several columns

c = ones(length(Rme))
x = [c Rme]

b = x\y
println("OLS coeffs according to Julia")
printmat(b)

OLS coeffs according to Julia
    -0.504
     1.341



# Python

In [3]:
using PyCall
sm = pyimport("statsmodels.api")     #activate this package

PyObject <module 'statsmodels.api' from 'C:\\Miniconda3\\lib\\site-packages\\statsmodels\\api.py'>

In [4]:
resultsP = sm.OLS(y, x).fit()        #can use Python functions directly

println(resultsP.summary())

PyObject <class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.519
Model:                            OLS   Adj. R-squared:                  0.518
Method:                 Least Squares   F-statistic:                     416.2
Date:                Sun, 04 Jul 2021   Prob (F-statistic):           2.72e-63
Time:                        16:59:44   Log-Likelihood:                -1241.7
No. Observations:                 388   AIC:                             2487.
Df Residuals:                     386   BIC:                             2495.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

In [5]:
println(keys(resultsP))              #print all keys (field names)

[:HC0_se, :HC1_se, :HC2_se, :HC3_se, :_HCCM, :__class__, :__delattr__, :__dict__, :__dir__, :__doc__, :__eq__, :__format__, :__ge__, :__getattribute__, :__gt__, :__hash__, :__init__, :__init_subclass__, :__le__, :__lt__, :__module__, :__ne__, :__new__, :__reduce__, :__reduce_ex__, :__repr__, :__setattr__, :__sizeof__, :__str__, :__subclasshook__, :__weakref__, :_abat_diagonal, :_cache, :_data_attr, :_data_in_cache, :_get_robustcov_results, :_is_nested, :_use_t, :_wexog_singular_values, :aic, :bic, :bse, :centered_tss, :compare_f_test, :compare_lm_test, :compare_lr_test, :condition_number, :conf_int, :conf_int_el, :cov_HC0, :cov_HC1, :cov_HC2, :cov_HC3, :cov_kwds, :cov_params, :cov_type, :df_model, :df_resid, :diagn, :eigenvals, :el_test, :ess, :f_pvalue, :f_test, :fittedvalues, :fvalue, :get_influence, :get_prediction, :get_robustcov_results, :initialize, :k_constant, :llf, :load, :model, :mse_model, :mse_resid, :mse_total, :nobs, :normalized_cov_params, :outlier_test, :params, :predic

In [6]:
b_P = resultsP.params                #the numerical results are now a Julia vector

println("Comparing the estimates in Julia and Python")
printmat([b b_P])

Comparing the estimates in Julia and Python
    -0.504    -0.504
     1.341     1.341



In [7]:
#we can run blocks of code like this, notice: $x and $y
py"""
  import numpy as np
  xx = np.matmul(np.matrix.transpose($x),$x)
  xy = np.matmul(np.matrix.transpose($x),$y)
  b_p = np.linalg.solve(xx,xy)
  #b_p = np.linalg.lstsq($x,$y,rcond=None)
  print(b_p)
  """

[-0.5041626   1.34104865]


# R

In [8]:
ENV["R_HOME"]="C:/PROGRA~1/R/R-40~1.4"    #path to R, do R.home() in R to see
using RCall                               #do import Pkg; Pkg.build("RCall") after changing R location 

In [9]:
@rput x y                          #send x and y to R

resultsR = reval("summary(mod <- lm(y ~ x-1))")  #run R code and print output
println(resultsR)

resultsR = reval("mod <- lm(y ~ x-1)")  #run R code

println(names(resultsR))           #print all keys (field names)

RObject{VecSxp}

Call:
lm(formula = y ~ x - 1)

Residuals:
    Min      1Q  Median      3Q     Max 
-17.981  -3.131  -0.359   2.281  52.361 

Coefficients:
   Estimate Std. Error t value Pr(>|t|)    
x1 -0.50416    0.30483  -1.654    0.099 .  
x2  1.34105    0.06573  20.401   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 5.954 on 386 degrees of freedom
Multiple R-squared:  0.5194,	Adjusted R-squared:  0.5169 
F-statistic: 208.6 on 2 and 386 DF,  p-value: < 2.2e-16


[:coefficients, :residuals, :effects, :rank, Symbol("fitted.values"), :assign, :qr, Symbol("df.residual"), :xlevels, :call, :terms, :model]


In [10]:
b_R = rcopy(resultsR[:coefficients])  #the numerical results are now a Julia array

println("Comparing the estimates in Julia and R")
printmat([b b_R[:,1]])

Comparing the estimates in Julia and R
    -0.504    -0.504
     1.341     1.341



In [11]:
#we can run blocks of code like this
#do @rput x y or use $x and $y in the code below

#solve x'x*b = x'y
R"""
xx <- t(x)%*%x
xy <- t(x)%*%y
b_R <- solve(xx,xy)
"""

RObject{RealSxp}
           [,1]
[1,] -0.5041626
[2,]  1.3410486


# C

This section illustrates some simple examples of how to call a C function. The functions are in the file `My_C_Stuff.c` which contains:

In [12]:
println(read("Data/My_C_Stuff.c",String))

#include <stddef.h>

// calculate the inner (dot) product of vectors Y and Y, returns the result (Sxy)
double c_dot(size_t n, double *Y, double *X) {
    double Sxy = 0.0;
    for (size_t i = 0; i < n; ++i) {
        Sxy += X[i]*Y[i];
    }
    return Sxy;
}

// calculate a simple regression, Y = a + b*X + u, puts (a,b) in vector ab, returns nothing
void c_ols(size_t n, double *Y, double *X, double *ab) {
    double Sx = 0.0, Sy = 0.0, Sxx = 0.0, Sxy = 0.0;
    for (size_t i = 0; i < n; ++i) {
        Sx  += X[i];
        Sy  += Y[i];
        Sxx += X[i]*X[i];
        Sxy += X[i]*Y[i];
    }
    ab[1] = (Sxy-Sx*Sy/n)/(Sxx-Sx*Sx/n);   //slope
    ab[0] = (Sy - ab[1]*Sx)/n;             //intercept
}


To compile to a dynamlic library (dll on windows), I use gcc (for x86_64) from [mingw-64](http://mingw-w64.org)
and run the following in the mingw terminal
```
gcc -shared -fPIC My_C_Stuff.c -o My_C_Stuff.dll
```

To call the C functions, place the dll file in the current folder and then run the following cells.

In [13]:
mylibc = "My_C_Stuff.dll"

b_c = zeros(2)          #where C will store the regression results
x2  = x[:,2]
@ccall mylibc.c_ols(length(y)::Csize_t, y::Ptr{Float64}, x2::Ptr{Float64}, b_c::Ptr{Float64})::Cvoid
println("Comparing the estimates in Julia and C")
printmat([b b_c])

Comparing the estimates in Julia and C
    -0.504    -0.504
     1.341     1.341



In this call `mylibc.c_ols` is the library.function, `length(y)::Csize_t` is the first input and its type (an `Int` indicating the number of elements in `y`), `y::Ptr{Float64}` is the second input and its type (an `Array{Float64}`)  and similarly for the remaining inputs. `Cvoid` is the type of the output, which here indicates that the function does not have an output. Rather, the function modifies the vector `b_c` by putting the OLS results there. 

We could potentially wrap this in a Julia function that checks for the right input types and outputs the `b_c`  vector.

In the next example, we instead use a function that outputs a `Float64` number. The function calculates the inner product of two vectors.

In [14]:
z = @ccall mylibc.c_dot(length(y)::Csize_t, y::Ptr{Float64}, x2::Ptr{Float64})::Float64

printlnPs("The inner product of x2 and y in Julia and C:  ",x2'y," ",z)

The inner product of x2 and y in Julia and C:   11071.648           11071.648
